In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [28]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
171910,CA,F,2005,Yazmeen,6
198955,CA,F,2012,Raya,20
310991,CA,M,1988,Deepak,5
114434,CA,F,1990,Destiney,12
32136,CA,F,1953,Ronnie,12


## Goal 1: Find the most popular baby name in California in 2018

In [29]:
babynames.loc[babynames["Year"] == 2018, ["Name"]].value_counts().index[0]

('Journey',)

## Goal 2: Baby names that start with j. 

In [38]:
babynames[babynames["Name"].str.startswith("J")]

,State,Sex,Year,Name,Count,name_length
16,CA,F,1910,Josephine,66,9
44,CA,F,1910,Jean,35,4
46,CA,F,1910,Jessie,32,6
59,CA,F,1910,Julia,28,5
66,CA,F,1910,Juanita,25,7
...,...,...,...,...,...,...
400567,CA,M,2021,Jovin,5,5
400568,CA,M,2021,Jr,5,2
400569,CA,M,2021,Juanjose,5,8
400570,CA,M,2021,Juel,5,4


## Goal 3: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [33]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].map(len)

#add that series to the dataframe as a column
babynames["name_length"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_length")

#drop that column
babynames_by_length = babynames_by_length.drop("name_length", axis=1)
babynames_by_length.head(5)

Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [20]:
babynames["name_length"] = babynames["Name"].str.len()
babynames.sort_values(["name_length"]).drop("name_length",axis=1, inplace=True)

Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [34]:
babynames.loc[babynames["Name"].str.len().sort_values().index]

,State,Sex,Year,Name,Count,name_length
376803,CA,M,2013,An,7,2
338135,CA,M,1999,Cj,6,2
264612,CA,M,1957,Ed,53,2
308634,CA,M,1987,Lo,6,2
356854,CA,M,2006,Aj,5,2
...,...,...,...,...,...,...
333732,CA,M,1997,Ryanchristopher,5,15
333556,CA,M,1997,Franciscojavier,5,15
318049,CA,M,1991,Ryanchristopher,7,15
310235,CA,M,1988,Franciscojavier,10,15


How does this work exactly? Let's break it into pieces.

In [35]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

376803    2
338135    2
264612    2
308634    2
356854    2
Name: Name, dtype: int64

In [36]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

Int64Index([376803, 338135, 264612, 308634, 356854, 308613, 295333, 321001,
            249345, 338172,
            ...
            308802, 313143, 340695, 318170, 330421, 333732, 333556, 318049,
            310235, 102498],
           dtype='int64', length=400762)

In [37]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture!
babynames.loc[index_sorted_by_length].head(5)

,State,Sex,Year,Name,Count,name_length
376803,CA,M,2013,An,7,2
338135,CA,M,1999,Cj,6,2
264612,CA,M,1957,Ed,53,2
308634,CA,M,1987,Lo,6,2
356854,CA,M,2006,Aj,5,2
